In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt 
import io
import sys
import os

In [18]:
EPOCH = 10
BATCH_SIZE = 128
LR = 0.001

In [3]:
T = transforms.ToTensor()
train_data = datasets.CIFAR10(root='~/newStuFile/ZHX/dataset/', train=True,transform=T,download=True)
test_data =datasets.CIFAR10(root='~/newStuFile/ZHX/dataset/',train=False,transform=T,download=True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

In [19]:
sys.path.append('..')
from alexnet import alexnet_cifar
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset=train_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)
test_loader = DataLoader(dataset=test_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)
model = alexnet_cifar().cuda()

In [20]:
len(train_loader)

391

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=LR)

In [22]:
from tqdm import tqdm
with tqdm(total = EPOCH*len(train_loader)) as pbar:
    for epoch in range(EPOCH):
        for i,data in enumerate(train_loader):
            inputs,labels = data
            inputs,labels = inputs.cuda(),labels.cuda()
            outputs = model(inputs)
            loss = criterion(outputs,labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pbar.update(1)
            pbar.desc = '%f'%loss.item()

0.844474: 100%|██████████| 3910/3910 [07:12<00:00,  9.05it/s]


In [24]:
torch.save(model,'../pts/cifar_alexnet.pt')